# Object Tracking with YOLO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import matplotlib.pyplot as plt
import cv2
import random, os
import numpy as np
from glob import glob

In [4]:
config = "/content/drive/MyDrive/Colab Notebooks/yolo_files/yolov3.cfg"
weights = "/content/drive/MyDrive/Colab Notebooks/yolo_files/yolov3.weights"
labels_path = "/content/drive/MyDrive/Colab Notebooks/yolo_files/coco.names"

with open(labels_path, 'r') as f:
    classes = f.read().strip().split('\n')

In [5]:
video_path = '/content/drive/MyDrive/Colab Notebooks/yolo_files/car_flow.mp4'

output_path = 'object_tracking_yolov3.mp4'

In [6]:
def load_yolo_model(config_path, weights_path):
    net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]
    return net, output_layers

def detect_objects(net, frame, output_layers):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    return boxes, confidences, class_ids


def draw_labels(boxes, confidences, class_ids, classes, frame):
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {round(confidence, 2)}", (x, y + 30), font, 2, color, 2)

net, output_layers = load_yolo_model(config, weights)


cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'XVID')

out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    boxes, confidences, class_ids = detect_objects(net, frame, output_layers)
    draw_labels(boxes, confidences, class_ids, classes, frame)
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Object Tracking with Ultranalytics

In [ ]:
# !pip install ultralytics

In [9]:
import cv2
import numpy as np
from ultralytics import YOLO

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

In [ ]:
!yolo track source='/content/drive/MyDrive/Colab Notebooks/yolo_files/car_flow.mp4' save=True

In [11]:
video_path = '/content/drive/MyDrive/Colab Notebooks/yolo_files/car_flow.mp4'

output_path = "object_tracking_yolov8.mp4"

In [12]:
model = YOLO("yolov8n.pt")

names = model.model.names

track_history = defaultdict(lambda: [])

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

result = cv2.VideoWriter(output_path,
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):

                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))

                if len(track) > 30:
                    track.pop(0)

                # Plot tracks
                points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break


result.release()
cap.release()
cv2.destroyAllWindows()


from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Object Counting

In [ ]:
# !pip install ultralytics

In [5]:
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import cv2

In [6]:
video_path = '/content/drive/MyDrive/Colab Notebooks/yolo_files/people.m4v'

output_path = "people_counting_yolov8.mp4"

In [7]:
region_points = [
    (359, 138),
    (359, 388),
    (409, 388),
    (409, 138)
]

In [8]:
model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter(output_path,
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))


counter = object_counter.ObjectCounter()

counter.set_args(view_img=True,
                 reg_pts=region_points,
                 classes_names=model.names,
                 draw_tracks=True)


while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    tracks = model.track(im0, persist=True, show=False)

    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)


cap.release()
video_writer.release()
cv2.destroyAllWindows()

from google.colab import files
files.download(output_path)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.

0: 480x640 3 persons, 1 car, 1 truck, 7.7ms
Speed: 2.2ms preprocess, 7.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 7.1ms
Speed: 2.3ms preprocess, 7.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 7.3ms
Speed: 2.2ms preprocess, 7.3ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 7.0ms
Speed: 2.4ms preprocess, 7.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 8.2ms
Speed: 2.2ms preprocess, 8.2ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 6.9ms
Speed: 2.2ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 7.5ms
Speed: 2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
import shutil, os
from glob import glob

In [10]:
output_path = '/content/people_counting_yolov8.mp4'

In [11]:
drive_path = '/content/drive/MyDrive/Colab Notebooks/outputs/'

In [12]:
os.makedirs(drive_path, exist_ok=True)

In [13]:
shutil.copy(output_path, drive_path)

'/content/drive/MyDrive/Colab Notebooks/outputs/people_counting_yolov8.mp4'

In [14]:
os.listdir(drive_path)

['people_counting_yolov8.mp4']

# Saving Outputs from Colab to Drive

In [ ]:
import shutil, os
from glob import glob

output_path = '/content/people_counting_yolov8.mp4'  # Kaydetmek istediğiniz yerel dosyanın yolu
drive_path = '/content/drive/MyDrive/Colab Notebooks/outputs/'  # Google Drive'daki hedef dizin


In [ ]:
# Hedef dizin mevcut değilse oluşturun
os.makedirs(drive_path, exist_ok=True)

In [ ]:
# Dosyayı kopyalayın
shutil.copy(output_path, drive_path)

'/content/drive/MyDrive/Colab Notebooks/outputs/people_counting_yolov8.mp4'

In [ ]:
os.listdir(drive_path)

['people_counting_yolov8.mp4']